<a href="https://colab.research.google.com/github/EricRibeiroAlves/DeteccaoFakeNews/blob/main/Modelos/%5BAP_T%5D_TI_Modelo_Gen%C3%A9rico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import itertools
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from google.colab import drive

In [2]:
# Importação do Dataset ISOT Fake News Dataset

# Carregar os dados
drive.mount('/content/drive')
dat_fake = "/content/drive/MyDrive/Eng. Controle e Automação/8º Semestre/AP/dataset_FakeNews/Fake.csv"
dat_real = "/content/drive/MyDrive/Eng. Controle e Automação/8º Semestre/AP/dataset_FakeNews/True.csv"
dt_fake = pd.read_csv(dat_fake)
dt_real = pd.read_csv(dat_real)

dt_fake['label'] = 'fake'
dt_real['label'] = 'real'

dt = pd.concat([dt_fake, dt_real], ignore_index=True)
dt = dt[(dt['subject'] == 'News') | (dt['subject'] == 'worldnews')]

dt.shape

Mounted at /content/drive


(19195, 5)

In [3]:
i=dt.label.value_counts()

In [4]:
fig=go.Figure(data=[go.Bar(x=['real','fake'], y=i,textposition='auto')])

fig.show()

In [5]:
X_train, X_test, y_train, y_test=train_test_split(dt['text'], dt.label, test_size=0.2, random_state=7)

In [6]:
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)

In [7]:
dfidf_train=tfidf_vectorizer.fit_transform(X_train)
tfidf_test=tfidf_vectorizer.transform(X_test)

In [17]:
pac=PassiveAggressiveClassifier(max_iter=10)
pac.fit(dfidf_train,y_train)

PassiveAggressiveClassifier(max_iter=10)

In [18]:
y_pred=pac.predict(tfidf_test)

In [19]:
score=accuracy_score(y_test,y_pred)
print(f'Accuracy:{round(score*100,2)}%')

Accuracy:99.77%


In [20]:
# Matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)

# Exibir matriz de confusão
print("Matriz de Confusão:")
print(conf_matrix)

# Visualizar matriz de confusão com Plotly
categories = ['Real', 'Fake']
fig = go.Figure(
    data=go.Heatmap(
        z=conf_matrix,
        x=categories,
        y=categories,
        colorscale="Viridis",
        text=conf_matrix,
        texttemplate="%{text}"
    )
)
fig.update_layout(
    title="Matriz de Confusão",
    xaxis_title="Previsto",
    yaxis_title="Verdadeiro"
)
fig.show()


Matriz de Confusão:
[[1798    1]
 [   8 2032]]


In [21]:
# Relatório de classificação
report = classification_report(y_test, y_pred)
print("Relatório de Classificação:")
print(report)


Relatório de Classificação:
              precision    recall  f1-score   support

        fake       1.00      1.00      1.00      1799
        real       1.00      1.00      1.00      2040

    accuracy                           1.00      3839
   macro avg       1.00      1.00      1.00      3839
weighted avg       1.00      1.00      1.00      3839

